In [2]:
from tokenizer_ivar import *
x = filereader("test.txt")
print(x)

['Er', 'ontstaan', 'gezwellen', '(tumoren).', 'Hoewel', 'het', 'woord', "'tumor'", 'voor', 'patiënten', 'vaak', 'een', 'angstige', 'bijklank', 'heeft', 'betekent', 'het', 'niet', 'meer', 'of', 'minder', 'dan', "'zwelling'.", 'Een', 'tumor', 'kan', 'zowel', 'goed-', 'als', 'kwaadaardig', 'zijn.', 'Een', 'goedaardige', 'tumor', 'wordt', 'ook', 'wel', 'benigne', 'genoemd,', 'een', 'kwaadaardige', 'maligne.', 'Bij', 'kanker', 'is', 'er', 'sprake', 'van', 'maligne', 'tumoren.', 'Kankerweefsel', 'geneest', 'niet', 'goed', 'en', 'gaat', 'makkelijk', 'bloeden.', 'Bloedverlies,', 'bijvoorbeeld', 'bij', 'ontlasting,', 'urine,', 'uit', 'de', 'tepel', 'of', 'bij', 'hoesten,', 'is', 'een', 'van', 'de', 'belangrijke', 'vroege', 'waarschuwingssymptomen.', 'De', 'gezwellen', 'drukken', 'op', 'andere', 'structuren', 'en', 'belemmeren', 'daarvan', 'de', 'werking.', 'Bij', 'de', 'darm', 'kan', 'bijvoorbeeld', 'passage', 'van', 'voedsel', 'onmogelijk', 'worden;', 'vanuit', '-', 'door', 'zwellingen', 'gebl

In [48]:
def encoder(word_list, max_dict_len, min_freqwentie):
    token_dict = {}
    token_value_dict = {}
    counter = 1

    # start vocab
    for word_index in range(len(word_list)):
        current_word = word_list[word_index]
        for letter_index in range(len(current_word)):
            letter = current_word[letter_index]
            if token_value_dict.get(letter) == None:
                token_value_dict[letter] = 1
                token_dict[letter] = counter
                counter += 1
            else:
                current_value = token_value_dict.get(letter)
                new_value = current_value + 1
                token_value_dict[letter] = new_value
    # tokenize original words 1 toker per letter
    words_in_tokens = []
    for word_index in range(len(word_list)):
        current_word = word_list[word_index]
        word_in_tokens = []
        for letter_index in range(len(current_word)):
            current_letter = current_word[letter_index]
            new_token = token_dict.get(current_letter)
            word_in_tokens.append(new_token)
        words_in_tokens.append(word_in_tokens)
    # reverse key value
    reversed_token_dict = {}
    for key,value in token_dict.items():
        new_key = value
        new_value = key
        reversed_token_dict[new_key] = new_value


    def pair_merger(token_lists, tokens_dict):
        new_token_lists = []
        temp_dict = {}
        pair_dict = {}
        for token_list_index in range(len(token_lists)):
            current_token_list = token_lists[token_list_index]
            if len(current_token_list) < 2:
                continue
            else:
                end_token_index = 1
                token_list_range = len(current_token_list)-1
                for token_index in range(token_list_range):

                    first_token_string = tokens_dict.get(current_token_list[token_index])
                    first_token = current_token_list[token_index]
                    secon_token_string = tokens_dict.get(current_token_list[end_token_index])
                    second_token = current_token_list[end_token_index]
                    end_token_index += 1

                    candidate = first_token_string + secon_token_string
                    if temp_dict.get(candidate) == None:
                        temp_dict[candidate] = 1
                        pair_dict[candidate] = [first_token,second_token]
                    else:
                        current_value = temp_dict.get(candidate)
                        new_value = current_value + 1
                        temp_dict[candidate] = new_value

        key_list = []
        value_list = []
        for key,value in temp_dict.items():
            key_list.append(key)
            value_list.append(value)

        value_check = max(value_list)
        if value_check <min_freqwentie:
            return None, None
        
        higest_value_index = value_list.index(max(value_list))
        new_string_key = key_list[higest_value_index]

        old_key_list = []
        for key in tokens_dict.keys():
            old_key_list.append(key)

        new_token_key = max(old_key_list)+1
        tokens_dict[new_token_key] = new_string_key
        new_pair = pair_dict.get(new_string_key)
        
        for token_list_index in range(len(token_lists)):
            current_token_list = token_lists[token_list_index]
            temp_token_list = current_token_list.copy()
            end_index = 1
            for token_index in range(len(current_token_list)-1):
                first_token = current_token_list[token_index]
                second_token = current_token_list[end_index]

                if first_token is new_pair[0] and second_token is new_pair[1]:
                    temp_token_list[token_index] = new_token_key
                    temp_token_list[end_index] = None
                    end_index += 1
                else:
                    end_index += 1
            # remove nonetypes
            if None in (temp_token_list):
                temp_token_list.remove(None)
                while None in (temp_token_list):
                    temp_token_list.remove(None)
            new_token_lists.append(temp_token_list)

        return new_token_lists, tokens_dict

    result_token_list = words_in_tokens.copy()
    result_token_dict = reversed_token_dict.copy()
    max_len = 10
    # condense all words into tokens
    while max_len >= 2:
        if len(result_token_dict) == max_dict_len:
            break
        
        temp_list, temp_dict = pair_merger(result_token_list,reversed_token_dict)
        if temp_list is None:
            break

        result_token_list = temp_list
        result_token_dict = temp_dict
        temp = []
        for i in range(len(result_token_list)):
            current_token_list = result_token_list[i]
            temp.append(len(current_token_list))
        
        max_len = max(temp)
    

    return result_token_list,result_token_dict

new_tokens, new_dict = encoder(x,101,10)
print(new_dict)

{1: 'E', 2: 'r', 3: 'o', 4: 'n', 5: 't', 6: 's', 7: 'a', 8: 'g', 9: 'e', 10: 'z', 11: 'w', 12: 'l', 13: '(', 14: 'u', 15: 'm', 16: ')', 17: '.', 18: 'H', 19: 'h', 20: 'd', 21: "'", 22: 'v', 23: 'p', 24: 'i', 25: 'ë', 26: 'k', 27: 'b', 28: 'j', 29: 'f', 30: '-', 31: ',', 32: 'B', 33: 'K', 34: 'c', 35: 'y', 36: 'D', 37: ';', 38: 'A', 39: ':', 40: '/', 41: 'x', 42: 'en', 43: 'an', 44: 'er', 45: 'or', 46: 'in', 47: 'el', 48: 'st', 49: 'ij', 50: 'aa', 51: 'oe', 52: 'ing', 53: 'oor', 54: 'oed'}


In [32]:
def Encwriter(token_dict):
    with open("result.enc","w") as writer:
        for key,value in token_dict.items():
            line = str(key)+":"+str(value)+"\n"
            writer.write(line)
    return

Encwriter(new_dict)

In [39]:
test = "eetlust lust eet"
encoder(test.split())

([[11], [10], [7]],
 {1: 'e',
  2: 't',
  3: 'l',
  4: 'u',
  5: 's',
  6: 'ee',
  7: 'eet',
  8: 'lu',
  9: 'lus',
  10: 'lust',
  11: 'eetlust'})